## Set the enviroments.

#### Connect to slime.

In [1]:
(start-swank)

T

In [19]:
(ext:vfork-execvp (list (namestring (translate-logical-pathname #P"amber:bin;sqm"))
                             "-O"
                             "-i" "/tmp/sqm-input.txt"
                             "-o" "/tmp/sqm-output.out"))

0

#### Add paths.

In [3]:
(leap.core::clear-path)
(add-path *default-pathname-defaults*)
(add-path "~/Development/amber16/dat/leap/cmd/")
(add-path "~/Development/amber16/dat/leap/lib/")
(add-path "~/Development/amber16/dat/leap/parm/")
(add-path "~/Development/amber16/dat/leap/prep/")
(add-path "~/Development/amber16/dat/antechamber/")

(#P"/Users/tuj24515/Development/amber16/dat/antechamber/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/prep/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/parm/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/lib/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/cmd/"
 #P"/Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/")

#### Load the amber force field.

In [4]:
(load-atom-type-rules "ATOMTYPE_AMBER.DEF")

(#<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [5]:
(source "leaprc.protein.ff14SB")

SIMPLE-WARNING: 
add-atom-types doesn't do anything
SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


T

In [6]:
(load-atom-type-rules "water_amber.def")

(#<FORCE-FIELD water_amber.def> #<FORCE-FIELD frcmod.ff14SB>
 #<FORCE-FIELD parm10.dat> #<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [7]:
(:= *ff* (leap.core::merged-force-field))

*FF*

## Create topology and coordinate file for Alanine

#### Load the alanine from Chemdraw file.

In [8]:
(defparameter *chemagg* (cando:load-chem-draw-aggregate "alanine2.cdxml"))

*CHEMAGG*

In [9]:
(build-hydrogens *chemagg*)

NIL

In [10]:
(cando::show-aggregate *chemagg*)

A Jupyter Widget

In [38]:
(nglv::show-aggregate *chemagg*)

An error occurred of type: UNDEFINED-FUNCTION: 
  #<UNDEFINED-FUNCTION>


NIL

#### Gather atoms to the center.

In [11]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *chemagg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *chemagg*)))

C_0    S


NIL

#### Set the bounding box.

In [12]:
(chem:set-property *chemagg* :bounding-box '(50.0 50.0 50.0))

NIL

#### Assign atom types and change one of them for alanine atom type definition.

In [13]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

####  Assign force-field parameters and minimize the energy of the molecule.

In [14]:
(cando:jostle *chemagg*)

NIL

In [15]:
(defparameter *ee* (energy:minimize *chemagg*))

minimize-no-fail
======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-log(Alpha)--Dir-------------Energy-----------RMSforce
 minSDnP       0     1      -inf   0.0              0.000          14078.609
 minSDnP       0     2     -7.92   0.0        2736035.680           8093.121
DONE absolute force test:
forceRmsMag(4913.449622).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     3      -inf   0.0              0.000           4913.450
 minCGnP       0     4     -7.80  39.1        1630584.721           3993.000
 minCGnP       0     5     -6.92  35.7        1111617.658           2572.576
 minCGnP       0     6     -7.50  47.4         996432.768           2165.203
 minCGnP       0     7     -6.90  70.7         814725.910           4271.527
 minCGnP       0     8     -8.22  59.1         721795.348           3334.941
 minCGnP       0     9     -7.53  72.8         549298.280           5779.417
 minCGnP       0    10    -10.59  61.3   

*EE*

#### Visualize the minimized structure and demonstrate that the force field parameters lead to the construction of a chemically

#### reasonable structure.

In [35]:
(nglv::show-aggregate *chemagg*)

SIMPLE-PROGRAM-ERROR: 
The symbol NGLV::SHOW-AGGREGATE has no function bound to it


NIL

In [16]:
(cando::show-aggregate *chemagg*)

A Jupyter Widget

In [21]:
(charges:calculate-am1-bcc-charges *chemagg*)

0

In [44]:
(save-mol2 *chemagg* "alanine.mol2")

Saving matter to /Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/alanine.mol2


NIL

#### Assign atom types and change one of them for alanine atom type definition.

In [22]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

#### Save the topology file and coordinate file.

In [23]:
(leap.topology:save-amber-parm-format *chemagg* "chemagg.top" "chemagg.crd" :assign-types nil)

T

## Create topology and coordinate file for Alanine and water atoms.

#### Load Tip3pfbox water box.

In [47]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

#### Make agggregate of the water box.

In [48]:
(print ~tip3pfbox)


#<AGGREGATE NIL> 

#<AGGREGATE NIL>

In [49]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [43]:
(nglv::show-aggregate *solvent*)

SIMPLE-PROGRAM-ERROR: 
The symbol NGLV::SHOW-AGGREGATE has no function bound to it


NIL

#### Assign the atom types of the force field to the water box.

In [50]:
(assign-atom-types *solvent*)

NIL

#### Add Tip3p water molecules to the alanine.

In [51]:
(leap.solvate::tool-solvate-and-shell *chemagg* *solvent* '(10.0 10.0 10.0))

Total bounding box for atom centers:  29.912177327655d0 25.675470742219375d0 23.684818054620703d0


NIL

In [48]:
(nglv::show-aggregate *newagg*)

SIMPLE-PROGRAM-ERROR: 
The symbol NGLV::SHOW-AGGREGATE has no function bound to it


NIL

In [52]:
(show *chemagg*)

A Jupyter Widget

#### Save the topology and coordinate file. New aggregate contains water molecules and alanine.

In [53]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [54]:
(leap.topology:save-amber-parm-format *chemagg* "chemagg_water.top" "chemagg_water.crd" :assign-types nil)

T

#### Calculate each energy parameters.

In [31]:
(chem:get-energy (chem:get-stretch-component *e*))

1720.1189821006878d0

In [32]:
(chem:get-energy (chem:get-angle-component *e*))

0.30171081698364277d0

In [33]:
(chem:get-energy (chem:get-dihedral-component *e*))

5.456751705606037d0

In [34]:
(chem:get-energy (chem:get-nonbond-component *e*))

658.8024508583202d0

In [34]:
(apropos "superpose")

CHEM:ADD-SUPERPOSE-ATOM  Function
CHEM:CLEAR-SUPERPOSE-ATOMS  Function
CHEM:SET-FINAL-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-FINAL-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SUPERPOSE  Function
CANDO::SUPERPOSE-AGAINST-TEMPLATE  Function
CANDO::SUPERPOSE-ALL  Function
CHEM:SUPERPOSE-ALL-HEAVY-ATOMS  Function
CHEM:SUPERPOSE-ENGINE
CANDO::SUPERPOSE-ONE  Function
CHEM:SUPERPOSE-SELECTED-ATOMS
CHEM::SUPERPOSER


NIL

In [88]:
(:= w (show *chemagg*)) w

A Jupyter Widget

In [93]:
(nglv::picked w)

(("atom" ("index" . 20) ("residueIndex" . 2) ("resname" . "NIL_3")
  ("x" . 216.329) ("y" . 322.549) ("z" . -0.62) ("element" . "H")
  ("chainname" . "A") ("resno" . 3) ("serial" . 21) ("vdw" . 1.1)
  ("covalent" . 0.31) ("hetero" . 1) ("bfactor" . 0) ("altloc" . "")
  ("atomname" . "HH33")))

In [94]:
(defun distance (p1 p2) (geom:vlength (geom:v- p1 p2)))

DISTANCE

In [95]:
(:= *a* (let* ((picked (nglv::picked w))
       (atom (cdr (assoc "atom" picked :test #'string=)))
       (x (cdr (assoc "x" atom :test #'string=)))
       (y (cdr (assoc "y" atom :test #'string=)))
       (z (cdr (assoc "z" atom :test #'string=)))
       (pos (geom:vec x y z))
        resulta)
     (chem:map-atoms nil (lambda (a) 
                                 (when (< (distance pos (chem:get-position a)) 0.01) 
                                     (setf resulta a))) *chemagg*)
              resulta))

*A*

In [96]:
*a*

#<ATOM :HH33/:H>

In [97]:
(:= *p* (core:make-cxx-object 'chem:chem-info))

*P*

In [98]:
(chem:compile-smarts *p* "CC")

T

In [99]:
(chem:matches *p* *a*)

NIL